In [1]:
import sys
sys.path.append('../')

import soynlp
print(soynlp.__version__)

0.0.46


soynlp 0.0.46+ 에서는 soynlp.noun.LRNounExtractor 를 보완한 LRNounExtractor_v2 를 제공합니다. 

version 2 에서는 (1) 명사 추출의 정확성을 높였으며, (2) 합성명사의 인식이 가능합니다. 또한 (3) 명사의 빈도를 정확히 계산합니다.

사용법은 version 1 과 비슷합니다. train_extract 함수를 통하여 명사 점수를 계산할 수 있습니다. verbose mode 일 경우에는 학습 과정의 진행 상황이 출력됩니다.

더 자세한 version 1 과 2 의 차이점은 [lovit.github.io/nlp/2018/05/08/noun_extraction_ver2](https://lovit.github.io/nlp/2018/05/08/noun_extraction_ver2/) 에 적어뒀습니다.

In [2]:
from soynlp.utils import DoublespaceLineCorpus
from soynlp.noun import LRNounExtractor_v2

corpus_path = '2016-10-20-news'
sents = DoublespaceLineCorpus(corpus_path, iter_sent=True)

train(), extract() 를 따로 진행할 수 있습니다.

In [3]:
%%time
noun_extractor = LRNounExtractor_v2(verbose=True)
noun_extractor.train(sents)
nouns = noun_extractor.extract()

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=1260, neg=1185, common=12
[Noun Extractor] counting eojeols
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained.
[Noun Extractor] batch prediction was completed for 146445 words
[Noun Extractor] checked compounds. discovered 35044 compounds
[Noun Extractor] 86133 nouns (35044 compounds) with min count=1
[Noun Extractor] 71.69 % eojeols are covered
[Noun Extractor] flushing ... done
CPU times: user 24.7 s, sys: 460 ms, total: 25.2 s
Wall time: 25 s


이 과정은 한 번에 일어나는 경우가 많기 때문에 train_extract() 함수를 이용해도 됩니다.

version 2 에서는 soynlp.utils 의 EojeolCounter 와 LRGraph 를 이용합니다. 명사가 포함되었다고 인식된 어절의 빈도수 비율이 출력됩니다. 71.69 % 의 어절에 추출된 명사가 포함되어 있습니다. 총 86,133 개의 명사가 추출되었으며, 이 중 35,044 개는 복합명사입니다.

복합 명사의 경우에는 min_count 가 1 일 경우에도 탐지가 될 수 있도록 하였습니다. min_count 와 minimum_noun_score 는 extract, train_extract 함수의 argument 로 조절할 수 있습니다.

In [4]:
%%time
nouns = noun_extractor.train_extract(sents)
# nouns = noun_extractor.train_extract(sents, min_count=1, minimum_noun_score=0.3)

[Noun Extractor] counting eojeols
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained.
[Noun Extractor] batch prediction was completed for 146445 words
[Noun Extractor] checked compounds. discovered 35044 compounds
[Noun Extractor] 86133 nouns (35044 compounds) with min count=1
[Noun Extractor] 71.69 % eojeols are covered
[Noun Extractor] flushing ... done
CPU times: user 25.7 s, sys: 284 ms, total: 25.9 s
Wall time: 25.7 s


nouns 는 {str: NounScore} 형식의 dict 입니다. 추출된 명사 단어에 대한 빈도수와 명사 점수가 namedtuple 인 NounScore 로 저장되어 있습니다.

version 1 의 명사 추출기에서는 '뉴스'라는 left-side substring 의 빈도수를 명사의 빈도수로 이용하였습니다만, version 2 에서는 어절에서 '뉴스'가 실제로 명사로 이용된 경우만 카운팅 됩니다. '뉴스방송'과 같은 복합명사의 빈도수는 '뉴스'에 포함되지 않습니다.

In [5]:
nouns['뉴스']

NounScore(frequency=4319, score=1.0)

LRNounExtractor_v2._compounds_components 에는 복합 명사의 components 가 저장되어 있습니다. _compounds_components 는 {str:tuple of str} 형식입니다.

In [8]:
for word in ['두바이월', '두바이월드', '두바이월드센터', '연합뉴스', '연합뉴스자료사', '연합뉴스자료사진', '군사기술']:
    print(word, noun_extractor._compounds_components.get(word, None))

두바이월 None
두바이월드 None
두바이월드센터 ('두바이', '월드', '센터')
연합뉴스 None
연합뉴스자료사 None
연합뉴스자료사진 ('연합뉴스', '자료', '사진')
군사기술 ('군사', '기술')


In [6]:
list(noun_extractor._compounds_components.items())[:10]

[('잠수함발사탄도미사일', ('잠수함', '발사', '탄도미사일')),
 ('미사일대응능력위원회', ('미사일', '대응', '능력', '위원회')),
 ('글로벌녹색성장연구소', ('글로벌', '녹색성장', '연구소')),
 ('시카고옵션거래소', ('시카고', '옵션', '거래소')),
 ('대한민국특수임무유공', ('대한민국', '특수', '임무', '유공')),
 ('철도산업발전소위원회', ('철도', '산업발전', '소위원회')),
 ('포괄적핵실험금지조약', ('포괄적', '핵실험', '금지', '조약')),
 ('유엔북한인권사무소', ('유엔', '북한인권', '사무소')),
 ('교육문화위원장', ('교육', '문화', '위원장')),
 ('중앙투자심사위원회', ('중앙투자심사', '위원회'))]

복합 명사도 nouns 에 포함되어 출력됩니다.

In [9]:
nouns['두바이월드센터']

NounScore(frequency=7, score=1.0)

LRNounExtractor_v2.decompose_compound 는 입력된 str 가 복합 명사일 경우, 이를 단일 명사의 tuple 로 분해합니다.

In [8]:
noun_extractor.decompose_compound('두바이월드센터시카고옵션거래소')

('두바이', '월드', '센터', '시카고', '옵션', '거래소')

복합명사가 아닌 경우에는 길이가 1 인 tuple 로 출력됩니다. 

In [7]:
noun_extractor.decompose_compound('두바이월드센터시카고옵션거래소라라라라라')

('두바이월드센터시카고옵션거래소라라라라라',)

LRNounExtractor_v2 는 soynlp.utils 의 LRGraph 를 이용합니다. 데이터의 L-R 구조를 살펴볼 수 있습니다.

In [9]:
noun_extractor.lrgraph.get_r('아이오아이')

[('', 123),
 ('의', 47),
 ('는', 40),
 ('와', 18),
 ('가', 18),
 ('에', 7),
 ('에게', 6),
 ('까지', 2),
 ('랑', 2),
 ('부터', 1)]

topk=10 으로 설정되어 있습니다. topk < 0 으로 설정하면 모든 R set 이 출력됩니다.

In [11]:
noun_extractor.lrgraph.get_r('아이오아이', topk=-1)

[('', 123),
 ('의', 47),
 ('는', 40),
 ('와', 18),
 ('가', 18),
 ('에', 7),
 ('에게', 6),
 ('까지', 2),
 ('랑', 2),
 ('부터', 1),
 ('도', 1),
 ('와의', 1),
 ('와는', 1),
 ('로', 1),
 ('를', 1),
 ('에서', 1)]

L-R 구조의 L parts 도 확인할 수 있습니다. 이 역시 topk=10 으로 기본값이 설정되어 있습니다.

In [10]:
noun_extractor.lrgraph.get_l('었다고')

[('있', 125),
 ('없', 76),
 ('만들', 37),
 ('늘', 32),
 ('맺', 29),
 ('열', 28),
 ('들', 19),
 ('입', 16),
 ('되', 14),
 ('줄', 14)]